In [33]:
import os
import pandas as pd
from datetime import datetime, timedelta
from pyspark.sql.types import StructType, StructField, IntegerType, StringType , FloatType
import sys
sys.path.append(r'/home/gabriel/Documents/proj/utils/')
from utils import createSparkSesion
from pyspark.sql import (functions as F,
                                        Window)
import matplotlib.pyplot as plt
import pandas as pd

In [2]:
spark = createSparkSesion()

25/05/20 23:12:45 WARN Utils: Your hostname, GabrielUnbunto resolves to a loopback address: 127.0.1.1; using 10.0.2.15 instead (on interface enp0s3)
25/05/20 23:12:45 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
25/05/20 23:12:45 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/05/20 23:12:46 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


<h1> Handlling Duplicates</h1>
Write a code to find and remove duplicate rows from a df imployees

In [3]:
pip install faker


Note: you may need to restart the kernel to use updated packages.


In [4]:
from faker import Faker
import random

In [5]:
fake = Faker('pt_br')
rand = random

In [6]:
#mocking some data
data   =  []
schema =  StructType([StructField("name", StringType(), False),
                      StructField("cpf", StringType(), False),
                      StructField("id", IntegerType(), False)])
for i in range(100):
   name = fake.name()
   cpf  = fake.cpf()
   id   = i
   if rand.random() >= 0.1 :
      data.append((name,cpf,id))
   else:
      data.append((name,cpf,id))
      data.append((name,cpf,id))


In [7]:
employees_df = spark.createDataFrame(data,schema = schema)

In [8]:
#solving easy mode kkk
employees_df_solution_1 = employees_df.distinct()

print(f'deduped : {employees_df_solution_1.count()}      ;      original:{employees_df.count()}')

deduped : 100      ;      original:106


In [9]:
#alternative mode , could be usefull in other cenarioes , such as , if  nao all the column having the exact same value

dedup_window = Window.partitionBy("name","cpf").orderBy("name")

employees_df_solution_2  = (employees_df
                              .withColumn("rn", F.row_number().over(dedup_window))
                              .filter(F.col("rn") == 1))

print(f'deduped : {employees_df_solution_2.count()}      ;      original:{employees_df.count()}')

deduped : 100      ;      original:106


<h1>NUll value handling</h1>
how would you replace a NULL value in the salary column of the employees table with the average salary

In [40]:
#mocking some data
data   =  []
schema =  StructType([StructField("name", StringType(), False),
                      StructField("cpf", StringType(), False),
                      StructField("id", IntegerType(), False),
                      StructField("salary", FloatType(), True)])
for i in range(100):
   name = fake.name()
   cpf  = fake.cpf()
   id   = i
   salary = rand.uniform(1000 , 10000)
   if rand.random() >= 0.1 :
      data.append((name,cpf,id,salary))
   else :
      data.append((name,cpf,id,None))

In [46]:
null_value_handling_df = spark.createDataFrame(data,schema=schema)

In [ ]:
#solving

avg_salary  = null_value_handling_df.select(F.avg("salary")).collect()[0][0]

null_value_handling_df_solution_1 = null_value_handling_df.withColumn("new_salary",F.when(F.col("salary").isNull(),avg_salary).otherwise(F.col("salary")))

null_value_handling_df_solution_1.filter(F.col("salary").isNull()).takePandas(20)

,name,cpf,id,salary,new_salary
0,Pedro Machado,927.130.658-95,28,None,4724.345528
1,Emilly Nogueira,806.315.947-01,49,None,4724.345528
2,Vinicius Araújo,065.218.947-49,67,None,4724.345528
3,José Campos,598.417.603-39,74,None,4724.345528
4,Isaque Machado,501.729.863-03,76,None,4724.345528
5,Sra. Beatriz Cunha,849.731.052-79,85,None,4724.345528
6,Dra. Laura das Neves,812.940.753-14,92,None,4724.345528
